<a href="https://colab.research.google.com/github/deepakmoud/Signature-Verification/blob/master/Signature_Verification_Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install -U keras


Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.3.1)


In [0]:
# Importing the Keras libraries and packages
import keras
from keras.layers import Input, Flatten, Dense
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.callbacks import Callback, ModelCheckpoint
from keras.applications.resnet_v2 import ResNet152V2
from keras.callbacks.callbacks import EarlyStopping
from keras.layers import Dropout
import warnings 
warnings.filterwarnings('ignore')
img_width, img_height = 128, 128

Using TensorFlow backend.


In [0]:
resnet15_model=keras.applications.resnet_v2.ResNet152V2(include_top=False, weights='imagenet', input_tensor=None, input_shape=(img_height,img_width,3), pooling=None, classes=1000)

print("pretrained Network is loaded")

pretrained Network is loaded


In [0]:
# Freeze the layers
for layer in resnet15_model.layers:
    layer.trainable = False
print("Pretrained layers are freezed")

Pretrained layers are freezed


In [0]:
model = Sequential()
model.add(resnet15_model)
#add fully connected layer:
model.add(Flatten())
model.add(Dense(128, activation='relu', name='hidden_layer1'))
model.add(keras.layers.Dropout(0.2))
model.add(Dense(128, activation='relu', name='hidden_layer2'))
model.add(keras.layers.Dropout(0.2))
model.add(Dense(64, activation='relu', name='classification_layer'))
keras.callbacks.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=1, mode='auto', baseline=None, restore_best_weights=True)
output_layer=model.add(Dense(1, activation='sigmoid', name='output_layer')) 
print("All layers top of pretrained layers are developed")

All layers top of pretrained layers are developed


In [0]:
train_data_dir ='/content/drive/My Drive/Dataset1/training_set'
val_data_dir ='/content/drive/My Drive/Dataset1/testing_set'
model_weights_file = 'resnet152v2_weights_tf_dim_ordering_tf_kernels_notop.h5'
nb_train_samples = 2117
nb_val_samples = 563
nb_epochs = 35
print("Input parameters are assigned")

Input parameters are assigned


In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Model is Complied")

Model is Complied


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=32,shuffle=False, class_mode='binary')
validation_generator = test_datagen.flow_from_directory(val_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=32,shuffle=False,class_mode='binary')

NameError: ignored

In [0]:
import os
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
folder = '/content/drive/My Drive/Dataset1/training_set/'
photos, labels = list(), list()
for r, d, f in os.walk(folder):
    for file in f:
        if ".png" in file:
          if(os.path.join(r, file)=='/content/drive/My Drive/Dataset1/training_set/genuine/{}'.format(file)):
            output=1
          elif(os.path.join(r, file)=='/content/drive/My Drive/Dataset1/training_set/forged/{}'.format(file)):
            output=0
        labels.append(output)
labels_train=np.array(labels)
print(os.path.join(r, file))
print(labels_train.shape)
labels_train=np.expand_dims(labels, axis=1)
print(labels_train)
print(labels_train.shape)

/content/drive/My Drive/Dataset1/training_set/genuine/006_01.PNG
(2117,)
[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]
(2117, 1)


In [0]:
import os
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
folder = '/content/drive/My Drive/Dataset1/testing_set/'
photos, labels = list(), list()
for r, d, f in os.walk(folder):
    for file in f:
        if ".png" in file:
          if(os.path.join(r, file)=='/content/drive/My Drive/Dataset1/testing_set/genuine/{}'.format(file)):
            output=1
          elif(os.path.join(r, file)=='/content/drive/My Drive/Dataset1/testing_set/forged/{}'.format(file)):
            output=0
        labels.append(output)
labels_test=np.array(labels)
print(os.path.join(r, file))
print(labels_test.shape)
labels_test=np.expand_dims(labels_test, axis=1)

print(labels_test.shape)

/content/drive/My Drive/Dataset1/testing_set/genuine/NFI-03001030.PNG
(563,)
(563, 1)


In [0]:
filepath="weights.best.hdf5"
callbacks = [ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')]

history = model.fit_generator( train_generator, callbacks = callbacks, nb_epoch=nb_epochs, validation_data=validation_generator)

print('Training Completed!')

Epoch 1/35
 2/67 [..............................] - ETA: 12:53 - loss: 4.8702 - accuracy: 0.0156

KeyboardInterrupt: ignored

In [0]:
 # save model and architecture to single file
model.save("/content/drive/My Drive/model_resnet_signature_verification.h5")
model.summary()
print("Saved model to disk")


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet152v2 (Model)          (None, 4, 4, 2048)        58331648  
_________________________________________________________________
flatten_1 (Flatten)          (None, 32768)             0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 128)               4194432   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
hidden_layer2 (Dense)        (None, 128)               16512     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
classification_layer (Dense) (None, 64)               

In [0]:
img_path = '/content/drive/My Drive/Dataset1/training_set/genuine/NFI-00102001.png'
label = ['Genuine','Forged']
img = image.load_img(img_path, target_size=(128,128))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
print(x.shape)
features = model.predict(x)
print(features)
print(features[0][0])
  
if features[0]>0.5:
    print(" is  Genuine")
else:
    print(" is Forged")
    

In [0]:
import h5py
with hf = h5py.File('model_resnet_signature_verification.h5', 'w')


hf.close()

In [0]:
from keras.models import load_model

model = load_model('model_resnet_signature_verification.h5')

In [0]:
model.save_weights("/content/drive/My Drive/model_resnet_weights_signature_verification.h5")